In [ ]:
!pip install ipynb --upgrade

In [2]:
from ipynb.fs.full.NER import get_entities
import pandas as pd 
import datetime
import requests
from collections import Counter
import glob

### Load csv

In [65]:
#df_clean = pd.read_csv('data_files/wallstreetbets_21.csv', lineterminator='\n')
#data = pd.read_csv('data_files/wallstreetbets_21.csv')
#data

,Unnamed: 0,comment_id,created,fullname,parent_id,subreddit,text,post_id,title
0,0,gz117qx,2021-05-22 05:37:10,t1_gz117qx,t1_gz10ocl,wallstreetbets,"Boohoo, go read a children’s bed time story wh...",NaN,NaN
1,1,gz1187q,2021-05-22 05:37:20,t1_gz1187q,t1_gz10shp,wallstreetbets,!GUH,NaN,NaN
2,2,gz11882,2021-05-22 05:37:21,t1_gz11882,t1_gz1187q,wallstreetbets,#[GUH](https://www.reddit.com/r/wallstreetbets...,NaN,NaN
3,3,gz11897,2021-05-22 05:37:21,t1_gz11897,t1_gz10wyw,wallstreetbets,In all fairness the streets are where I get my...,NaN,NaN
4,4,gz1189c,2021-05-22 05:37:21,t1_gz1189c,t1_gz10u2g,wallstreetbets,Why,NaN,NaN
...,...,...,...,...,...,...,...,...,...
9996,9996,gz2wmte,2021-05-22 18:29:45,t1_gz2wmte,t1_gz2wjg9,wallstreetbets,Making money here is like being paid to be the...,NaN,NaN
9997,9997,gz2wn05,2021-05-22 18:29:48,t1_gz2wn05,t3_ni1fw5,wallstreetbets,Hype is the most interesting of all fundamenta...,NaN,NaN
9998,9998,gz2wn4h,2021-05-22 18:29:49,t1_gz2wn4h,t3_niory4,wallstreetbets,Oh wow they went up and down... again....\n\nS...,NaN,NaN
9999,9999,gz2wn6r,2021-05-22 18:29:50,t1_gz2wn6r,t3_nimruw,wallstreetbets,"I always loved the stock long term, we are loo...",NaN,NaN


In [75]:
old_filenames = []
old_filenames.append('wallstreetbets_44.csv')

In [77]:
# read all csv files
for file_name in glob.glob('DATA/reddit/'+'*.csv'):
    print('* '+file_name)
    if file_name not in old_filenames:
        data = pd.read_csv(file_name,lineterminator='\n')
        # Transform data
        print('   * Transforming data')
        dfs = transform_reddit_data(data)
        df_submissions = dfs[0]
        df_comments = dfs[1]
        # Persist submissions and comments 
        print('   * Persisting submissions')
        insert_submissions(df_submissions)
        print('   * Persisting comments')
        insert_comments(df_comments)
        old_filenames.append(file_name)

* DATA/reddit/Stocks_3.csv
* DATA/reddit/Stocks_3.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/wallstreetbets_52.csv
* DATA/reddit/wallstreetbets_52.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/wallstreetbets_46.csv
* DATA/reddit/wallstreetbets_46.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/wallstreetbets_5.csv
* DATA/reddit/wallstreetbets_5.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/Stocks_15.csv
* DATA/reddit/Stocks_15.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/Stocks_29.csv
* DATA/reddit/Stocks_29.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/Stocks_28.csv
* DATA/reddit/Stocks_28.csv
   * Transforming data
   * Persisting submissions
   * Persisting comments
* DATA/reddit/Stocks_14.csv
* DA

KeyError: "['post_id'] not found in axis"

----

## Transforming the data

In [60]:
# Returns list with submissions dataframe and comments dataframe
def transform_reddit_data(data: pd.DataFrame):
        dfs = []
        # submissions
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_submissions = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_submissions = data.drop(['Unnamed: 0', 'fullname'],axis=1)
        df_submissions['created'] = df_submissions['created'].apply(datetime_to_date)
        df_submissions = df_submissions.dropna()
        df_submissions.rename(columns = {'post_id':'id'}, inplace = True)
        df_submissions['Organizations'] = df_submissions['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_submissions)
        # comments
        #candidates =['Unnamed: 0','comment_id', 'fullname', 'parent_id', 'title']
        #df_comments = data.drop([x for x in candidates if x in data.columns], axis=1)
        df_comments = data.drop(['Unnamed: 0', 'fullname', 'post_id'],axis=1)
        df_comments['created'] = df_comments['created'].apply(datetime_to_date)
        df_comments = df_comments.dropna()
        df_comments['Organizations'] = df_comments['text'].apply(get_entities).apply(clean_orgs)
        dfs.append(df_comments)
        
        return dfs

In [6]:
# Changes utc timestamp to datetime.date
def datetime_to_date(timestamp):
    return pd.to_datetime(timestamp).date()

In [7]:
# Mentions that we are interested in
selected_orgs =  ['HKG', 'Alibaba','AMC', 'Palantir Technologies', 'PLTR', 'FORD', 'Lordstown Motors', 'RIDE', 'Virgin Galactic', 'SPCE', 'AI', 'C3.AI', 'TSLA', 'GE', 'GME', 'AAPL', 'Tesla', 'Apple', 'General Electric', 'GE', 'NOK', 'Nokia']
orgs_dict = {"Alibaba":"HKG","AMC":"AMC","Palantir Technologies":"PLTR","FORD":"FORD", "Lordstown Motors":"RIDE","Virgin Galactic":"SPCE","c3.AI":"AI","Tesla":"TSLA", "General Electric":"GE","Apple":"AAPL","GameStop":"GME","Gamestop":"GME", "Nokia":"NOK"}


In [8]:
# returning a list of mentioned tickers
def clean_orgs(organizations):
    orgs = []
    for org in organizations:
        if org in selected_orgs:
            if org in orgs_dict:
                org = orgs_dict[org]
                orgs.append(org)
            else: 
                orgs.append(org)
    for org in orgs:
        o = set(orgs)
        orgs = list(o)
    return orgs

In [9]:
clean_orgs(['Virgin Galactic'])

['SPCE']

## A Look at the mentions

In [24]:
# Creates a list of mentionend entities

#data['Organizations'] = data['text'].apply(get_entities)
#orgs = data['Organizations'].to_list()
#orgs_flat = [org for sublist in orgs for org in sublist] # Pulls out entities from the nested lists in orgs => new flat list
# Print 20 most mentions ORGs
#from collections import Counter
#org_freq = Counter(orgs_flat)
#org_freq.most_common(20)                                                       

NameError: name 'data' is not defined

## Transform and save the data to database

In [49]:
#submissionsransforming into lists of submissions and comments
dfs = transform_reddit_data(data)
submissions = dfs[0]
comments = dfs[1]

In [52]:
data

,Unnamed: 0,comment_id,created,fullname,parent_id,subreddit,text
0,0,gzo4vsv,2021-05-27 19:02:58,t1_gzo4vsv,t1_gzo3wbw,wallstreetbets,Yes. Is it smart? Debatable.
1,1,gzo4vv6,2021-05-27 19:02:58,t1_gzo4vv6,t1_gzo4lej,wallstreetbets,put a majority of my account into this based o...
2,2,gzo4vvp,2021-05-27 19:02:59,t1_gzo4vvp,t1_gzo4r94,wallstreetbets,We are in this togheter brother🦍
3,3,gzo4vwz,2021-05-27 19:02:59,t1_gzo4vwz,t3_nm414o,wallstreetbets,CLOV!
4,4,gzo4vyx,2021-05-27 19:03:00,t1_gzo4vyx,t3_nm414o,wallstreetbets,Now do for PLTR what you did for AMC
...,...,...,...,...,...,...,...
9996,9996,gzoh41e,2021-05-27 20:30:39,t1_gzoh41e,t3_nmg57b,wallstreetbets,SPCE baby girls. I need some hooker money for...
9997,9997,gzoh45k,2021-05-27 20:30:41,t1_gzoh45k,t1_gzogdpm,wallstreetbets,I thought about that today too. It has differe...
9998,9998,gzoh461,2021-05-27 20:30:41,t1_gzoh461,t1_gzogoih,wallstreetbets,It's been in the $10 range for months tho
9999,9999,gzoh470,2021-05-27 20:30:41,t1_gzoh470,t3_nmehx2,wallstreetbets,You got balls buddy! I spend another 1k on amc...


In [10]:
## SUBMISSIONS subset
#print(len(submissions))
#df = submissions[1:500]
#insert_submissions(df)
#COMMENTS subset
#print(len(comments))
#df_c = comments[500:5500]
#
#insert_comments(df_c)

#insert_c(df_c)

NameError: name 'comments' is not defined

In [19]:
url = 'http://localhost:5050/'

In [11]:
def insert_submissions(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        
        #create submission nodes
        url2 = 'submission/'+row['subreddit']+'/'+row['created'].strftime('%m-%d-%Y')+'/'+row['id']
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('submission id '+ row['id'] + ' failed insertion')
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/submission/'+row['id']+'/'+ org
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mentions between '+ row['id'] + ' and ' + org + ' failed creation')

In [12]:
def insert_comments(dataframe: pd.DataFrame):
    
    for index, row in dataframe.iterrows():
        
        #create comment nodes
        url2 = 'comment/'+row['subreddit']+'/'+row['created'].strftime('%m-%d-%Y')+'/'+row['comment_id']+'/'+row['parent_id']
        response = requests.post(url+url2)
        if not response.status_code == 200:
            print ('comment id '+ row['comment_id'] + ' failed insertion')
            
        # create mentions relationships
        for org in row['Organizations']:
            url3 = 'connection/mentions/comment/'+row['comment_id']+'/'+ org
            response = requests.post(url+url3)
            if not response.status_code == 200:
                print ('mention between '+ row['comment_id'] + ' and ' + org + ' failed creation')
    
           
        # create replies relationships
        row['parent_id'] = pd.Series(row['parent_id'], dtype="string") #turning parent_id from type Series to String
        for parent_id in row['parent_id']:
            p_id = parent_id[3:]
            if parent_id[:2] == 't1':
                url_reply = 'connection/replies/comment/' + row['comment_id'] + '/' + p_id
            elif parent_id[:2] == 't3':
                url_reply = 'connection/replies/submission/' + row['comment_id'] + '/' + p_id
            response = requests.post(url+url_reply)
            if not response.status_code == 200:
                print ('replies relationship between '+ row['comment_id'] + ' and ' + p_id + ' failed creation')